In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import os
import re

# Specify the path to the existing database file
db_path = r"..Relational Database Design\DEL 5 - 2\D5\Final - 02\womenwordcup2023_module5.db"

# Check if the database file exists
if not os.path.exists(db_path):
    messagebox.showerror("Error", f"Database file {db_path} does not exist. Please check the path.")
    exit(1)

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

def validate_date(date_str):
    if re.match(r'\d{2}/\d{2}/\d{4}', date_str):
        return True, ""
    else:
        return False, "Invalid date format. Please use MM/DD/YYYY."

def validate_non_negative_integer(value):
    try:
        if int(value) >= 0:
            return True, ""
        else:
            return False, "Value must be a non-negative integer."
    except ValueError:
        return False, "Value must be an integer."

def validate_match_result(result):
    if re.match(r'^[A-Za-z\s]+ \d+ _ [A-Za-z\s]+ \d+$', result):
        return True, ""
    else:
        return False, "Invalid result format. Please use 'Team A Score _ Team B Score' (e.g., 'Germany 2 _ USA 3')."

def validate_non_empty_string(value):
    if isinstance(value, str) and value.strip() and not value.strip().isdigit():
        return True, ""
    else:
        return False, "Value must be a non-empty string and not just a number."

def record_exists(table, column, value):
    cursor.execute(f"SELECT 1 FROM {table} WHERE {column} = ?", (value,))
    return cursor.fetchone() is not None

def create_game(match_id, match_date, attendance, result):
    valid_match_id, error_match_id = validate_non_negative_integer(match_id)
    valid_date, error_date = validate_date(match_date)
    valid_attendance, error_attendance = validate_non_negative_integer(attendance)
    valid_result, error_result = validate_match_result(result)
    
    if not (valid_match_id and valid_date and valid_attendance and valid_result):
        messagebox.showerror("Error", f"Invalid input data: {error_match_id} {error_date} {error_attendance} {error_result}")
        return
    
    if record_exists('Game', 'MatchID', match_id):
        messagebox.showerror("Error", "A game with this Match ID already exists.")
        return

    cursor.execute("INSERT INTO Game (MatchID, Match_Date, Attendance, Result) VALUES (?, ?, ?, ?)", 
                   (match_id, match_date, attendance, result))
    conn.commit()
    messagebox.showinfo("Success", "Record added successfully.")

def create_team(team_id, team_name, win, lose, draw):
    valid_team_id, error_team_id = validate_non_negative_integer(team_id)
    valid_team_name, error_team_name = validate_non_empty_string(team_name)
    valid_win, error_win = validate_non_negative_integer(win)
    valid_lose, error_lose = validate_non_negative_integer(lose)
    valid_draw, error_draw = validate_non_negative_integer(draw)
    
    if not (valid_team_id and valid_team_name and valid_win and valid_lose and valid_draw):
        messagebox.showerror("Error", f"Invalid input data: {error_team_id} {error_team_name} {error_win} {error_lose} {error_draw}")
        return
    
    if record_exists('Team', 'TeamID', team_id):
        messagebox.showerror("Error", "A team with this Team ID already exists.")
        return

    cursor.execute("INSERT INTO Team (TeamID, Team_Name, Win, Lose, Draw) VALUES (?, ?, ?, ?, ?)", 
                   (team_id, team_name, win, lose, draw))
    conn.commit()
    messagebox.showinfo("Success", "Record added successfully.")

def create_manager(manager_id, manager_first_name, manager_last_name):
    valid_manager_id, error_manager_id = validate_non_negative_integer(manager_id)
    valid_first_name, error_first_name = validate_non_empty_string(manager_first_name)
    valid_last_name, error_last_name = validate_non_empty_string(manager_last_name)
    
    if not (valid_manager_id and valid_first_name and valid_last_name):
        messagebox.showerror("Error", f"Invalid input data: {error_manager_id} {error_first_name} {error_last_name}")
        return
    
    if record_exists('Manager', 'ManagerID', manager_id):
        messagebox.showerror("Error", "A manager with this Manager ID already exists.")
        return

    cursor.execute("INSERT INTO Manager (ManagerID, Manager_First_Name, Manager_Last_Name) VALUES (?, ?, ?)", 
                   (manager_id, manager_first_name, manager_last_name))
    conn.commit()
    messagebox.showinfo("Success", "Record added successfully.")

def read_games():
    cursor.execute("SELECT * FROM Game")
    return cursor.fetchall()

def read_teams():
    cursor.execute("SELECT * FROM Team")
    return cursor.fetchall()

def read_managers():
    cursor.execute("SELECT * FROM Manager")
    return cursor.fetchall()

def update_game(match_id, match_date, attendance, result):
    valid_match_id, error_match_id = validate_non_negative_integer(match_id)
    valid_date, error_date = validate_date(match_date)
    valid_attendance, error_attendance = validate_non_negative_integer(attendance)
    valid_result, error_result = validate_match_result(result)
    
    if not (valid_match_id and valid_date and valid_attendance and valid_result):
        messagebox.showerror("Error", f"Invalid input data: {error_match_id} {error_date} {error_attendance} {error_result}")
        return
    
    if not record_exists('Game', 'MatchID', match_id):
        messagebox.showerror("Error", "No game with this Match ID exists.")
        return

    cursor.execute("UPDATE Game SET Match_Date = ?, Attendance = ?, Result = ? WHERE MatchID = ?", 
                   (match_date, attendance, result, match_id))
    conn.commit()
    messagebox.showinfo("Success", "Record updated successfully.")

def update_team(team_id, team_name, win, lose, draw):
    valid_team_id, error_team_id = validate_non_negative_integer(team_id)
    valid_team_name, error_team_name = validate_non_empty_string(team_name)
    valid_win, error_win = validate_non_negative_integer(win)
    valid_lose, error_lose = validate_non_negative_integer(lose)
    valid_draw, error_draw = validate_non_negative_integer(draw)
    
    if not (valid_team_id and valid_team_name and valid_win and valid_lose and valid_draw):
        messagebox.showerror("Error", f"Invalid input data: {error_team_id} {error_team_name} {error_win} {error_lose} {error_draw}")
        return
    
    if not record_exists('Team', 'TeamID', team_id):
        messagebox.showerror("Error", "No team with this Team ID exists.")
        return

    cursor.execute("UPDATE Team SET Team_Name = ?, Win = ?, Lose = ?, Draw = ? WHERE TeamID = ?", 
                   (team_name, win, lose, draw, team_id))
    conn.commit()
    messagebox.showinfo("Success", "Record updated successfully.")

def update_manager(manager_id, manager_first_name, manager_last_name):
    valid_manager_id, error_manager_id = validate_non_negative_integer(manager_id)
    valid_first_name, error_first_name = validate_non_empty_string(manager_first_name)
    valid_last_name, error_last_name = validate_non_empty_string(manager_last_name)
    
    if not (valid_manager_id and valid_first_name and valid_last_name):
        messagebox.showerror("Error", f"Invalid input data: {error_manager_id} {error_first_name} {error_last_name}")
        return
    
    if not record_exists('Manager', 'ManagerID', manager_id):
        messagebox.showerror("Error", "No manager with this Manager ID exists.")
        return

    cursor.execute("UPDATE Manager SET Manager_First_Name = ?, Manager_Last_Name = ? WHERE ManagerID = ?", 
                   (manager_first_name, manager_last_name, manager_id))
    conn.commit()
    messagebox.showinfo("Success", "Record updated successfully.")

def delete_game(match_id):
    valid_match_id, error_match_id = validate_non_negative_integer(match_id)
    if not valid_match_id:
        messagebox.showerror("Error", f"Invalid input data: {error_match_id}")
        return
    
    if not record_exists('Game', 'MatchID', match_id):
        messagebox.showerror("Error", "No game with this Match ID exists.")
        return

    cursor.execute("DELETE FROM Game WHERE MatchID = ?", (match_id,))
    conn.commit()
    messagebox.showinfo("Success", "Record deleted successfully.")

def delete_team(team_id):
    valid_team_id, error_team_id = validate_non_negative_integer(team_id)
    if not valid_team_id:
        messagebox.showerror("Error", f"Invalid input data: {error_team_id}")
        return
    
    if not record_exists('Team', 'TeamID', team_id):
        messagebox.showerror("Error", "No team with this Team ID exists.")
        return

    cursor.execute("DELETE FROM Team WHERE TeamID = ?", (team_id,))
    conn.commit()
    messagebox.showinfo("Success", "Record deleted successfully.")

def delete_manager(manager_id):
    valid_manager_id, error_manager_id = validate_non_negative_integer(manager_id)
    if not valid_manager_id:
        messagebox.showerror("Error", f"Invalid input data: {error_manager_id}")
        return
    
    if not record_exists('Manager', 'ManagerID', manager_id):
        messagebox.showerror("Error", "No manager with this Manager ID exists.")
        return

    cursor.execute("DELETE FROM Manager WHERE ManagerID = ?", (manager_id,))
    conn.commit()
    messagebox.showinfo("Success", "Record deleted successfully.")

# GUI Application
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Database Manager")

        self.table_var = tk.StringVar()
        self.table_var.set("Game")

        self.create_widgets()

    def create_widgets(self):
        # Frame for table selection
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        tk.Label(frame, text="Select Table:").grid(row=0, column=0, padx=5)
        tables = ["Game", "Team", "Manager"]
        self.table_menu = ttk.Combobox(frame, textvariable=self.table_var, values=tables, state="readonly")
        self.table_menu.grid(row=0, column=1, padx=5)

        # Frame for buttons
        btn_frame = tk.Frame(self.root)
        btn_frame.pack(pady=10)

        self.add_btn = tk.Button(btn_frame, text="Add Record", command=self.add_record)
        self.add_btn.grid(row=0, column=0, padx=5)

        self.read_btn = tk.Button(btn_frame, text="Read Records", command=self.read_records)
        self.read_btn.grid(row=0, column=1, padx=5)

        self.update_btn = tk.Button(btn_frame, text="Update Record", command=self.update_record)
        self.update_btn.grid(row=0, column=2, padx=5)

        self.delete_btn = tk.Button(btn_frame, text="Delete Record", command=self.delete_record)
        self.delete_btn.grid(row=0, column=3, padx=5)

        # Frame for output
        self.output_frame = tk.Frame(self.root)
        self.output_frame.pack(pady=10)

    def add_record(self):
        table = self.table_var.get()
        if table == "Game":
            self.show_game_form("add")
        elif table == "Team":
            self.show_team_form("add")
        elif table == "Manager":
            self.show_manager_form("add")

    def read_records(self):
        table = self.table_var.get()
        self.clear_output_frame()
        records = []
        if table == "Game":
            records = read_games()
            tk.Label(self.output_frame, text="Match ID | Match Date | Attendance | Result").pack()
        elif table == "Team":
            records = read_teams()
            tk.Label(self.output_frame, text="Team ID | Team Name | Win | Lose | Draw").pack()
        elif table == "Manager":
            records = read_managers()
            tk.Label(self.output_frame, text="Manager ID | First Name | Last Name").pack()

        for record in records:
            tk.Label(self.output_frame, text=str(record)).pack()

    def update_record(self):
        table = self.table_var.get()
        if table == "Game":
            self.show_game_form("update")
        elif table == "Team":
            self.show_team_form("update")
        elif table == "Manager":
            self.show_manager_form("update")

    def delete_record(self):
        table = self.table_var.get()
        if table == "Game":
            self.show_delete_form("Game")
        elif table == "Team":
            self.show_delete_form("Team")
        elif table == "Manager":
            self.show_delete_form("Manager")

    def show_game_form(self, action):
        self.clear_output_frame()
        form = GameForm(self.output_frame, action)
        form.pack()

    def show_team_form(self, action):
        self.clear_output_frame()
        form = TeamForm(self.output_frame, action)
        form.pack()

    def show_manager_form(self, action):
        self.clear_output_frame()
        form = ManagerForm(self.output_frame, action)
        form.pack()

    def show_delete_form(self, table):
        self.clear_output_frame()
        form = DeleteForm(self.output_frame, table)
        form.pack()

    def clear_output_frame(self):
        for widget in self.output_frame.winfo_children():
            widget.destroy()

class GameForm(tk.Frame):
    def __init__(self, parent, action):
        super().__init__(parent)
        self.action = action
        self.create_widgets()

    def create_widgets(self):
        tk.Label(self, text="Match ID:").grid(row=0, column=0, padx=5, pady=5)
        self.match_id_entry = tk.Entry(self)
        self.match_id_entry.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(self, text="Match Date (MM/DD/YYYY):").grid(row=1, column=0, padx=5, pady=5)
        self.match_date_entry = tk.Entry(self)
        self.match_date_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(self, text="Attendance:").grid(row=2, column=0, padx=5, pady=5)
        self.attendance_entry = tk.Entry(self)
        self.attendance_entry.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(self, text="Result:").grid(row=3, column=0, padx=5, pady=5)
        self.result_entry = tk.Entry(self)
        self.result_entry.grid(row=3, column=1, padx=5, pady=5)

        self.submit_btn = tk.Button(self, text="Submit", command=self.submit)
        self.submit_btn.grid(row=4, column=0, columnspan=2, pady=10)

    def submit(self):
        match_id = self.match_id_entry.get()
        match_date = self.match_date_entry.get()
        attendance = self.attendance_entry.get()
        result = self.result_entry.get()

        if self.action == "add":
            create_game(match_id, match_date, attendance, result)
        elif self.action == "update":
            update_game(match_id, match_date, attendance, result)
        elif self.action == "delete":
            delete_game(match_id)

class TeamForm(tk.Frame):
    def __init__(self, parent, action):
        super().__init__(parent)
        self.action = action
        self.create_widgets()

    def create_widgets(self):
        tk.Label(self, text="Team ID:").grid(row=0, column=0, padx=5, pady=5)
        self.team_id_entry = tk.Entry(self)
        self.team_id_entry.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(self, text="Team Name:").grid(row=1, column=0, padx=5, pady=5)
        self.team_name_entry = tk.Entry(self)
        self.team_name_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(self, text="Win:").grid(row=2, column=0, padx=5, pady=5)
        self.win_entry = tk.Entry(self)
        self.win_entry.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(self, text="Lose:").grid(row=3, column=0, padx=5, pady=5)
        self.lose_entry = tk.Entry(self)
        self.lose_entry.grid(row=3, column=1, padx=5, pady=5)

        tk.Label(self, text="Draw:").grid(row=4, column=0, padx=5, pady=5)
        self.draw_entry = tk.Entry(self)
        self.draw_entry.grid(row=4, column=1, padx=5, pady=5)

        self.submit_btn = tk.Button(self, text="Submit", command=self.submit)
        self.submit_btn.grid(row=5, column=0, columnspan=2, pady=10)

    def submit(self):
        team_id = self.team_id_entry.get()
        team_name = self.team_name_entry.get()
        win = self.win_entry.get()
        lose = self.lose_entry.get()
        draw = self.draw_entry.get()

        if self.action == "add":
            create_team(team_id, team_name, win, lose, draw)
        elif self.action == "update":
            update_team(team_id, team_name, win, lose, draw)
        elif self.action == "delete":
            delete_team(team_id)

class ManagerForm(tk.Frame):
    def __init__(self, parent, action):
        super().__init__(parent)
        self.action = action
        self.create_widgets()

    def create_widgets(self):
        tk.Label(self, text="Manager ID:").grid(row=0, column=0, padx=5, pady=5)
        self.manager_id_entry = tk.Entry(self)
        self.manager_id_entry.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(self, text="First Name:").grid(row=1, column=0, padx=5, pady=5)
        self.first_name_entry = tk.Entry(self)
        self.first_name_entry.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(self, text="Last Name:").grid(row=2, column=0, padx=5, pady=5)
        self.last_name_entry = tk.Entry(self)
        self.last_name_entry.grid(row=2, column=1, padx=5, pady=5)

        self.submit_btn = tk.Button(self, text="Submit", command=self.submit)
        self.submit_btn.grid(row=3, column=0, columnspan=2, pady=10)

    def submit(self):
        manager_id = self.manager_id_entry.get()
        first_name = self.first_name_entry.get()
        last_name = self.last_name_entry.get()

        if self.action == "add":
            create_manager(manager_id, first_name, last_name)
        elif self.action == "update":
            update_manager(manager_id, first_name, last_name)
        elif self.action == "delete":
            delete_manager(manager_id)

class DeleteForm(tk.Frame):
    def __init__(self, parent, table):
        super().__init__(parent)
        self.table = table
        self.create_widgets()

    def create_widgets(self):
        tk.Label(self, text=f"Delete from {self.table}").grid(row=0, column=0, padx=5, pady=5)
        
        if self.table == "Game":
            tk.Label(self, text="Match ID:").grid(row=1, column=0, padx=5, pady=5)
            self.id_entry = tk.Entry(self)
            self.id_entry.grid(row=1, column=1, padx=5, pady=5)
        elif self.table == "Team":
            tk.Label(self, text="Team ID:").grid(row=1, column=0, padx=5, pady=5)
            self.id_entry = tk.Entry(self)
            self.id_entry.grid(row=1, column=1, padx=5, pady=5)
        elif self.table == "Manager":
            tk.Label(self, text="Manager ID:").grid(row=1, column=0, padx=5, pady=5)
            self.id_entry = tk.Entry(self)
            self.id_entry.grid(row=1, column=1, padx=5, pady=5)

        self.submit_btn = tk.Button(self, text="Delete", command=self.submit)
        self.submit_btn.grid(row=2, column=0, columnspan=2, pady=10)

    def submit(self):
        record_id = self.id_entry.get()
        
        if self.table == "Game":
            delete_game(record_id)
        elif self.table == "Team":
            delete_team(record_id)
        elif self.table == "Manager":
            delete_manager(record_id)

def main():
    root = tk.Tk()
    app = App(root)
    root.mainloop()

if __name__ == "__main__":
    main()

# Close the connection when the application exits
conn.close()
